[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/XPretrain-colab/blob/main/test.ipynb)

In [ ]:
!pip install datasets yt_dlp scenedetect[opencv]

import datasets, json
from yt_dlp import YoutubeDL

!mkdir /content/videos

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

dataset = datasets.load_dataset("camenduru/hdvila_test", split='train[:1%]')
# dataset = datasets.load_dataset("camenduru/microsoft-XPretrain", split='train[:1%]')
first_2_data = list(dataset)[:2]

for dump in first_2_data:
  json_string = json.dumps(dump)
  row = json.loads(json_string)
  video_id = row['video_id']
  print(video_id)
  try:
    download_video(video_id)
    clips = row["clip"]
    for clip in clips:
      video_path = f"/content/videos/{video_id}.mp4"
      # !scenedetect -i {video_path} -o /content/adaptive --drop-short-scenes detect-adaptive split-video
      !scenedetect -i {video_path} -o /content/content --drop-short-scenes detect-content split-video
      # !scenedetect -i {video_path} -o /content/threshold --drop-short-scenes detect-threshold split-video
  except:
    pass

In [ ]:
import os

def find_top_10_largest_files(folder_path):
    file_sizes = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            file_size = os.path.getsize(file_path)
            file_sizes.append((file_path, file_size))

    sorted_files = sorted(file_sizes, key=lambda x: x[1], reverse=True)
    top_10_files = sorted_files[:10]
    return top_10_files

folder_path = "/content/content"
top_10_files = find_top_10_largest_files(folder_path)
for file_path, file_size in top_10_files:
    print(f"File: {file_path} | Size: {file_size} bytes")